## Load Credentials from a JSON

In [1]:
import json
import tweepy
import re
from collections import Counter
import time


In [2]:
#Load Twitter Credentials File
with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    TWITTER_AUTH = tweepy.OAuthHandler(info['CONSUMER_KEY'], info['CONSUMER_SECRET'])
    TWITTER_AUTH.set_access_token(info['ACCESS_KEY'],info['ACCESS_SECRET'])

TWITTER = tweepy.API(TWITTER_AUTH)


In [3]:
async def async_get_user_interactions(search, outlist):
    twitter_user = TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                count=200,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended'
    )
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    b = b.lower()
    b = b.replace(search, "")
    out = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    outlist.append(out)
    
   
    


In [11]:
async def init_async_get_user_interactions(search, outlist):
    twitter_user = TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                count=200,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended'
    )
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    b = b.lower()
    b = b.replace(search, "")
    out = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    top = Counter(out).most_common(10)
    output = [person[0] for person in top]
    outlist.append(output)

## Trio Async Tests

## 1------------------

In [12]:
import trio

## 2 --------------

In [13]:
async def interaction_chain(first_user):
    start_time = time.time()
    all_interactions = []
    index = []
    index.append(first_user)
    

    await init_async_get_user_interactions(first_user, all_interactions)
    
    print(all_interactions)
    print("----------Entering 2nd level. Time so far:", time.time() - start_time)
    
    

    loop_num = 1
    async with trio.open_nursery() as nursery:
        for interaction in all_interactions :
            print("Loop # ", loop_num, " . Time so far:", time.time() - start_time)
            loop_num +=1
            user = interaction
            conns = []
            nursery.start(async_get_user_interactions, user, conns)
            index.append(user)
            all_interactions.append(conns)
        
    
    
    
    print("Total time:", time.time() - start_time)
    return all_interactions, index

In [14]:
interactions,index = trio.run(interaction_chain, 'austen')

[['lambdaschool', 'rjnance', 'johnnewcastle2', 'slouischarles', 'bitcoinbingbong', 'webdevmason', 'ohmaar1', 'danoherrin', 'popeyethe13th', 'merrill_r']]
----------Entering 2nd level. Time so far: 1.1543898582458496
Loop #  1  . Time so far: 1.1546423435211182
Loop #  2  . Time so far: 1.1547269821166992
Loop #  3  . Time so far: 1.154787540435791
Loop #  4  . Time so far: 1.1548264026641846
Loop #  5  . Time so far: 1.1548643112182617
Loop #  6  . Time so far: 1.1549015045166016
Loop #  7  . Time so far: 1.154942274093628
Loop #  8  . Time so far: 1.1549792289733887
Loop #  9  . Time so far: 1.1550168991088867
Loop #  10  . Time so far: 1.1550571918487549
Loop #  11  . Time so far: 1.1550941467285156
Loop #  12  . Time so far: 1.1551403999328613
Loop #  13  . Time so far: 1.1551828384399414
Loop #  14  . Time so far: 1.155217170715332
Loop #  15  . Time so far: 1.1553802490234375
Loop #  16  . Time so far: 1.1554698944091797
Loop #  17  . Time so far: 1.1555147171020508
Loop #  18  . 

/home/xailus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: coroutine 'Nursery.start' was never awaited


 1.355670690536499
Loop #  816  . Time so far: 1.3558857440948486
Loop #  817  . Time so far: 1.3559398651123047
Loop #  818  . Time so far: 1.356123685836792
Loop #  819  . Time so far: 1.3561773300170898
Loop #  820  . Time so far: 1.3562302589416504
Loop #  821  . Time so far: 1.3562822341918945
Loop #  822  . Time so far: 1.3563342094421387
Loop #  823  . Time so far: 1.3563861846923828
Loop #  824  . Time so far: 1.3564379215240479
Loop #  825  . Time so far: 1.356489658355713
Loop #  826  . Time so far: 1.3565411567687988
Loop #  827  . Time so far: 1.356593132019043
Loop #  828  . Time so far: 1.3566539287567139
Loop #  829  . Time so far: 1.356705904006958
Loop #  830  . Time so far: 1.356757640838623
Loop #  831  . Time so far: 1.356809377670288
Loop #  832  . Time so far: 1.3568649291992188
Loop #  833  . Time so far: 1.356917142868042
Loop #  834  . Time so far: 1.356968641281128
Loop #  835  . Time so far: 1.3570201396942139
Loop #  836  . Time so far: 1.3570716381072998
Lo

KeyboardInterrupt: 

In [19]:
interactions

[['lambdaschool',
  'rjnance',
  'johnnewcastle2',
  'slouischarles',
  'bitcoinbingbong',
  'webdevmason',
  'ohmaar1',
  'danoherrin',
  'popeyethe13th',
  'merrill_r'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [13]:
index

['austen',
 'lambdaschool',
 'rjnance',
 'johnnewcastle2',
 'slouischarles',
 'bitcoinbingbong',
 'webdevmason',
 'ohmaar1',
 'danoherrin',
 'popeyethe13th',
 'merrill_r']